In [1]:
import pandas as pd
import numpy as np
from bert_embedding import BertEmbedding

bert_embedding = BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_cased')

def bert_embedder(df):
    count = 0
    batch_size = 10
    return_list = []
    while count < len(df):
        bert_df = bert_embedding.embedding(df.iloc[count:min(count+batch_size,len(df))]['title'], oov_way='avg')
        for title in bert_df:
            sentence, word_vec = title
            word_vec = np.array(word_vec)
            return_list.append(np.mean(word_vec, axis=0))
            count += 1
        if count % batch_size == 0:
            print(str(count) + "/" + str(len(df)) +" completed")
    return return_list

In [2]:
df = pd.read_csv('nasdaq_articles.csv', index_col=0)
df.head()

,title,description,return,sentiment
3/20/2020 0:26,Apple Is Back In The Buying Zone,Apple currently trading slightly below fair va...,-3.79,1.0
3/20/2020 2:51,Stock futures jump as tumultuous week comes to...,Central banks have taken measures to buffer th...,-3.79,1.0
3/20/2020 3:13,Oncopeptides Provides Update Regarding COVID-1...,Oncopeptides AB (Nasdaq Stockholm: ONCO) annou...,-3.79,1.0
3/20/2020 3:24,"US stock futures gain, dollar snaps eight-day ...",Asian stocks and US futures gained and the dol...,-3.79,1.0
3/20/2020 3:49,Immunovia Publishes the Annual Report for the ...,Immunovia has today published the annual repor...,-3.79,0.0


In [3]:
# Binary Classification
# when stock gain is more than 1% then it is a positive market signal
df['market_signal'] = np.where(df['return']>=0, 1, 0)

In [ ]:
# when stock gain is more than 1% then it is a positive market signal
df['market_signal'] = np.where(df['return']>=1, 1, 0)
# when stock loss is more than 1% then it is a negative market signal
df['market_signal'] = np.where(df['return']<=-1, -1, df['market_signal'])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(['market_signal'],axis=1), df['market_signal'], test_size=0.2, random_state=777, shuffle=True)

In [5]:
# Use news article titles to perform sentiment analysis
X_train = bert_embedder(X_train)
X_test = bert_embedder(X_test)

10/798 completed
20/798 completed
30/798 completed
40/798 completed
50/798 completed
60/798 completed
70/798 completed
80/798 completed
90/798 completed
100/798 completed
110/798 completed
120/798 completed
130/798 completed
140/798 completed
150/798 completed
160/798 completed
170/798 completed
180/798 completed
190/798 completed
200/798 completed
210/798 completed
220/798 completed
230/798 completed
240/798 completed
250/798 completed
260/798 completed
270/798 completed
280/798 completed
290/798 completed
300/798 completed
310/798 completed
320/798 completed
330/798 completed
340/798 completed
350/798 completed
360/798 completed
370/798 completed
380/798 completed
390/798 completed
400/798 completed
410/798 completed
420/798 completed
430/798 completed
440/798 completed
450/798 completed
460/798 completed
470/798 completed
480/798 completed
490/798 completed
500/798 completed
510/798 completed
520/798 completed
530/798 completed
540/798 completed
550/798 completed
560/798 completed
5

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score

lreg = LogisticRegression(C=3)
lreg.fit(X_train, y_train)

C:\Users\junha\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
y_pred = lreg.predict(X_test)
accuracy = accuracy_score(y_pred,y_test)
f1 = f1_score(y_pred,y_test)

In [38]:
accuracy

0.59